In [1]:
import pandas as pd
import numpy as np
import os
import operator
import requests
import geopy.geocoders as geocode
from datetime import date

# Handling

In [ ]:
df = pd.read_csv("Data/Yellowpages_05-05-2020.csv")
#df2 = pd.read_csv("Data/Googlemaps_20-04-2020.csv")

In [ ]:
df = pd.concat([df, df2], sort=False)

In [ ]:
def merge_sheets(path = "Data/"):
    files = os.listdir(path)
    
    for file in files:
        initial = True
        if file[-3:] == "csv":
            if initial == True:
                df = pd.read_csv(path + file)
                initial = False
            else:
                this_df = pd.read_csv(path + file)
                df = pd.concat([df, this_df], sort=False)
    return df

def cleanup(df):
    df = df.drop(labels="Unnamed: 0", axis=1) 
            
    df = df.reset_index()
    df = df.drop(labels="index", axis=1)
    
    for i in range(0, len(df)):
        # only applies for google maps!
        #if df.iloc[i, 1][-11:] == ", Australia":
        #    df.iloc[i, 1] = df.iloc[i, 1][:-11]
            
        if df["Category"][i].find("-") != -1:
            df["Category"][i] = df["Category"][i][:df["Category"][i].find("-")]
            
        tel = str(df.Phone[i])
        if tel[0] == "1":
            tel = "+6" + tel
        else:
            bracket_o = tel.find("(")
            if bracket_o != -1:
                tel = tel[:bracket_o] + tel[bracket_o+2:]
                bracket_c = tel.find(")")
                if bracket_c != -1:
                    tel = tel[:bracket_c] + tel[bracket_c+1:]
                if str(tel[:3]) != "+61":
                    tel = "+61" + tel
            elif tel[0] == "0":
                tel = "+61" + tel[1:]
        df.Phone[i] = tel
            
    return df

def get_zip(df):
    zip_codes = list()
    state = list()
    for i in range(0, len(df.Name)):
        try:
            zip_code = df.Adress[i][-4:]
            zip_codes.append(zip_code)
        except:
            zip_codes.append("N/V")
            
        try:
            intzip_code = int(zip_code)
            if intzip_code >= 1000 and intzip_code < 3000:
                state.append("New South Wales")
            elif (intzip_code >= 3000 and intzip_code < 4000) or (intzip_code >= 8000 and intzip_code < 9000):
                state.append("Victoria")
            elif (intzip_code >= 4000 and intzip_code < 5000) or (intzip_code >= 9000 and intzip_code < 10000):
                state.append("Queensland")
            elif intzip_code >= 5000 and intzip_code < 6000:
                state.append("South Australia")
            elif intzip_code >= 6000 and intzip_code < 7000:
                state.append("Western Australia")
            elif intzip_code >= 7000 and intzip_code < 8000:
                state.append("Tasmania")
            elif intzip_code >= 800 and intzip_code < 1000:
                state.append("Nothern Australia")
            else:
                state.append("N/V")
        except:
            state.append("N/V")
            
            
    df["zip"] = zip_codes
    df["state"] = state
    return df

"""    for name in duplicates["names"]:
        part_df = df[df.Name == name]
        for i in range(0, len(part_df)):
            try:
                if len(part_df[part_df.zip == part_df.zip[i]]) > 1:
                    print("zip here.")
            except:
                pass"""

def find_duplicates(df):
    cutOff = int((len(df)/2) + 1)
    duplicates = list()
    for i in range(0, cutOff):
        if len(df[df.Name == df.Name[i]]) > 1:
            duplicates.append(df.Name[i])
    
    return duplicates

def handle_duplicates(df, duplicates):
    columns = ["Adress", "GeoLocation", "Phone", "Website", "Email"]
    for i in range(0, len(duplicates)):
        part_df = df[(df.Name == df.Name[i]) & (df.zip == df.zip[i])]

        best_entry = dict()
        for column in columns:
            for i, entry in enumerate(part_df[column]):
                   if entry != "N/V" and entry != "":
                        if i not in best_entry.keys():
                            best_entry[i] = 1
                        else:
                            best_entry[i] = best_entry[i] + 1

        best_entry = max(best_entry.items(), key=operator.itemgetter(1))[0]
        empty_columns = list()


        for column in columns:
            try:
                if part_df[column][best_entry] == "N/V":
                    empty_columns.append(column)
            except:
                pass

        if len(empty_columns) != 0:
            for i in range(0, len(part_df)):
                if i != best_entry:
                    for column in empty_columns:
                        if part_df[column][i] != "N/V" and part_df[column][i] != "":
                              part_df[column][best_entry] =  part_df[column][i]

        df = df.drop(index=part_df.index)
        df = df.append(part_df.iloc[best_entry])
    return df

In [ ]:
#df = merge_sheets()

In [ ]:
df = cleanup(df)

In [ ]:
df = get_zip(df)

In [ ]:
duplicates = find_duplicates(df)

In [ ]:
df.tail(3)

In [ ]:
def try_websites(df):
    for i in range(0, len(df.Name)):
        if i % 100 == 0:
            print(i)
        try:
            request = requests.get(str(df.Website[i]))
        except:
            df.Website[i] = "N/V"
    return df

In [ ]:
df.to_excel(f"Data/Data_{date.today().strftime('%d-%m-%Y')}.xlsx")

# Geocoding

In [2]:
from opencage.geocoder import OpenCageGeocode
from pprint import pprint

key = 'd223aasd************21efbb'
geocoder = OpenCageGeocode(key)

df = pd.read_excel("Data/TestDataGeo.xlsx")
df = df.drop("Unnamed: 0", axis=1)

In [56]:
def handleGeocoding(df):
    for i in range(0, len(df)):
        location = geocoder.geocode(df.Adress[i])
        geoCode = f"{location[0]['geometry']['lat']},{location[0]['geometry']['lng']}"
        df["GeoLocation"][i] = geoCode
    return df

def geoCodeToArray(df):
    dataAsArray = df.to_numpy()
    geoIndex = df.columns.get_loc("GeoLocation")
    for i in range(0, len(dataAsArray)):
        value = dataAsArray[i][geoIndex]
        if type(value) is str:
            dec = value.find(",")
            if dec != -1:
                dataAsArray[geoIndex][i] = [value[0:dec],value[dec+1:]]
    return pd.DataFrame(data=dataAsArray, columns=df.columns)
        
def convertCropCode(data):
    codeDict = {1: "Fruit Farm", 2: "Berry Farm", 3: "Vineyard or Winery", 4: "Vegetable Farm", 5: "Nut Farm", 6: "Wheat & Grain Farm", \
               7: "Animal & Animal Products Facility", 8: "Fish & Seafood Farm", 9: "Tree Planting or Forestry Facilitiy"}
    cropsIndex = data.columns.get_loc("Crops")
    categoryIndex = data.columns.get_loc("Category")
    dataAsArray = data.to_numpy()
    for i in range(0, len(dataAsArray)):
        crop = dataAsArray[i][cropsIndex]
        if type(crop) is int:
            dataAsArray[i][categoryIndex] = codeDict[crop]
        
    return pd.DataFrame(data=dataAsArray, columns=data.columns)

In [51]:
df = df[df['Adress'].notnull()]
df = df.reset_index()

In [57]:
df = handleGeocoding(df)

TypeError: list expected at most 1 arguments, got 2

In [3]:
df.head()

,level_0,index,Index,Name,Adress,GeoLocation,Phone,Website,Category,Email,Crops,Source,DateUpdated,DateChecked,zip,state,Multiple Locations
0,0,0,16454,Burke & Wills Winery,"89 Coombe Ln, Mia Mia VIC 3444","-37.2444,144.4515",+613 5425 5400,http://www.wineandmusic.net/,Vineyard or Winery,wineandmusic@bigpond.com,3,https://www.yellowpages.com.au/search/listings...,05/05/2020,2020-06-01,3444,Victoria,NaN
1,1,1,16456,Wild Dog Winery,"6 Farrington Close, Warragul VIC 3820","-38.1924149,145.9268202",+614 06 003 602,http://www.wilddogwinery.com/,Vineyard or Winery,luke@wilddogwinery.com,3,https://www.yellowpages.com.au/search/listings...,05/05/2020,2020-06-01,3820,Victoria,NaN
2,2,2,16458,Mia Valley Estate,"203 Daniels Lane, Mia Mia VIC 3444","-36.9954573,144.554238",+614 07311895,http://www.miavalleyestate.com.au/,Vineyard or Winery,info@miavalleyestate.com.au,3,https://www.yellowpages.com.au/search/listings...,05/05/2020,2020-06-01,3444,Victoria,NaN
3,3,3,16459,Wild Duck Creek Estate,"764 Spring Flat Rd, Heathcote VIC 3523","-36.9410711,144.7188642",+613 5433 3133,http://www.wildduckcreekestate.com.au/,Vineyard or Winery,winery@wdce.com.au,3,https://www.yellowpages.com.au/search/listings...,05/05/2020,2020-06-01,3523,Victoria,NaN
4,4,4,16461,Meehan Vineyard,"4536 McIvor Hwy, Heathcote VIC 3523","-36.8738005,144.6346631",+614 38 981 847,http://www.meehanvineyard.com/,Vineyard or Winery,meehanvineyard@bigpond.com,3,https://www.yellowpages.com.au/search/listings...,05/05/2020,2020-06-01,3523,Victoria,NaN


In [58]:
df = convertCropCode(df)
df = df[df['Name'].notnull()]

In [60]:
df.to_excel("Data/TestDataGeo.xlsx")